In [ ]:
!pip install upbitpy

In [ ]:
from upbitpy import Upbitpy
import datetime
import logging
import time

INTERVAL_MIN = 5

def Wait(min):
    Now = datetime.datetime.now()
    Second = 60 - Now.second
    Second += 60 * (min - (Now.minute % min + 1))
    time.sleep(Second)


In [ ]:
# 요청 회수 제한 방지
def CheckTimeCandle(Upbit):
    RemainCandles =Upbit.get_remaining_req()
    if (RemainCandles is None or  'candles' not in RemainCandles.keys()):
        return
    if int(RemainCandles['candles']['sec']) == 0:
        logging.debug('>>> sleep 1 seconds')
        time.sleep(1)

In [ ]:
# 메인 기능
def main():
    Upbit = Upbitpy()

    # 원화 가져오기
    Markets = Upbit.get_market_all()
    KrwMarkets = [M['market'] for M in Markets if M['market'].startswith('KRW')]

    # 7일 거래량
    for M in KrwMarkets:
        Candles7d[M] = Upbit.get_weeks_candles(M, count=1)[0]
        CheckTimeCandle(Upbit)
    while True:
        logging.info('평균 거래량 대비 {}분 거래량 비율========================'.format(INTERVAL_MIN))
        for M in KrwMarkets:
            Vol = Upbit.get_minutes_candles(1, M, count=1)[0]['candle_acc_trade_volume']
            Vol7d =  Candles7d[M]['candle_acc_trade_volume']
            Vol7dAvg = (((Vol7d/7.0)/24.0)/60.0)*INTERVAL_MIN
            VolRatio = format((Vol/Vol7dAvg)*100.0, '.2f')
            logging.info('[{}] {}% (거래량:{}, 평균:{})'.format(
                        M, VolRatio, format(Vol, '.2f'), format(Vol7dAvg, '.2f')))
        Wait(INTERVAL_MIN)

if __name__ =='__main__':
    logging.basicConfig(level=logging.INFO)
    main()